In [2]:
import torch
import torchvision
from PIL import Image

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

In [4]:
image, target = train_dataset[1]
image.show()